In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from category_encoders import MEstimateEncoder, TargetEncoder
from sklearn.inspection import permutation_importance

import seaborn as sns
import shap

import lightgbm as lgb

import sys
sys.path.append('../lightgbm')

C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(al

In [17]:
# File Paths
test_df_path = '../../Dataset/test.csv/test.csv'
process_train_path = './processed/processed_train_2.csv'
process_train_path = './processed/dropped_train.csv'
processed_test_path = './processed/processed_test_2.csv'
feats_path = './best_feats/Onehot_only_mean_NANs_modified_prices.csv'
output_path = './output/lgbm_pred_topFeat.csv'

new_df_path = './yh/processed_train.csv'
new_test_path = './yh/processed_test.csv'

top_n = 75

In [18]:
feats_df = pd.read_csv(feats_path)
feats = feats_df['col_name'].values.tolist()[:top_n]
processed_df = pd.read_csv(process_train_path)
processed_df = processed_df[feats+['price_doc']]
X = processed_df.drop(['price_doc'], axis=1)
y = processed_df['price_doc']
X = X[feats]

new_df = pd.read_csv(new_df_path)

for item in ['max_floor', 'floor', 'build_year', 'num_room', 'kitch_sq', 'state']:
    X[item] = new_df[item]

In [19]:
test_df = pd.read_csv(test_df_path)
processed_test_df = pd.read_csv(processed_test_path)

new_test = pd.read_csv(new_test_path)
for item in ['max_floor', 'floor', 'build_year', 'num_room', 'kitch_sq', 'state']:
    processed_test_df[item] = new_test[item]


processed_test_df

,id,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,...,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_3.0,material_4.0,material_5.0,material_6.0,material_nan
0,30474,39.00,20.700000,2,9,1998.0,1,8.9,3.0,2.615514e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30475,79.20,34.404467,8,17,0.0,3,1.0,1.0,2.553630e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30476,40.50,25.100000,3,5,1960.0,2,4.8,2.0,9.946335e+06,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,30477,62.80,36.000000,17,17,2016.0,2,62.8,3.0,2.149409e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30478,40.00,40.000000,17,17,0.0,1,1.0,1.0,2.553630e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,38131,52.20,31.800000,10,12,1973.0,2,9.1,2.0,7.811375e+06,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7658,38132,54.09,34.404467,14,14,2016.0,2,0.0,1.0,5.299528e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7659,38133,41.08,1.000000,12,12,1.0,1,1.0,1.0,7.307411e+06,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7660,38134,34.80,19.800000,8,9,1977.0,1,6.4,2.0,7.128794e+06,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [20]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.metrics import mean_squared_error
import optuna

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "n_estimators": trial.suggest_int("n_estimators", 800, 1200),
#         "boosting": trial.suggest_categorical("boosting", ["gbdt", "rf", "dart"]),
        "lambda_l2": trial.suggest_float("lambda_l2", 0, 10),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0, 1),
        "bagging_freq": 1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

In [22]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2023-10-31 17:14:06,210] A new study created in memory with name: no-name-efd83a18-7ffe-4ce0-b5ea-8c3d1081bb10
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.8905949787278113, colsample_bytree=0.6344956525184016 will be ignored. Current value: feature_fraction=0.8905949787278113
[LightGBM] [Warning] min_data_in_leaf is set=24, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=24
[LightGBM] [Warning] bagging_fraction is set=0.16825019363096372, subsample=0.5144684959155901 will be ignored. Current value: bagging_fraction=0.16825019363096372
[LightGBM] [Warning] lambda_l2 is set=7.524539953525803, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.524539953525803
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-31 17:14:08,944] Trial 0 finished with value: 2076176.6831971689 and parameters: {'n_estimators': 1023, 'lambda_l2': 7.524539953525803, 'bagging_fraction': 0.16825019363096372, 'num_leaves': 722, 'feature_fraction': 0.8905949787278113, 'max_depth': 14, 'learning_rate': 0.08763532231321279, 'subsample': 0.5144684959155901, 'colsample_bytree': 0.6344956525184016, 'min_data_in_leaf': 24}. Best is trial 0 with value: 2076176.6831971689.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.6309170404311941, colsample_bytree=0.45794120091688734 will be ignored. Current value: feature_fraction=0.6309170404311941
[LightGBM] [Warning] min_data_in_leaf is set=46, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=46
[LightGBM] [Warning] bagging_fraction is set=0.7515737450446494, subsample=0.9617648657636173 will be ignored. Current value: bagging_fraction=0.7515737450446494
[LightGBM] [Warning] lambda_l2 is set=7.791222774316102, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.791222774316102
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-31 17:14:14,279] Trial 1 finished with value: 2675734.876848277 and parameters: {'n_estimators': 916, 'lambda_l2': 7.791222774316102, 'bagging_fraction': 0.7515737450446494, 'num_leaves': 387, 'feature_fraction': 0.6309170404311941, 'max_depth': 24, 'learning_rate': 0.0013163118604080995, 'subsample': 0.9617648657636173, 'colsample_bytree': 0.45794120091688734, 'min_data_in_leaf': 46}. Best is trial 0 with value: 2076176.6831971689.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


...

[I 2023-10-31 17:23:05,182] Trial 47 finished with value: 2039122.487092675 and parameters: {'n_estimators': 838, 'lambda_l2': 8.405039715441077, 'bagging_fraction': 0.8389716277112718, 'num_leaves': 495, 'feature_fraction': 0.5795923644697714, 'max_depth': 11, 'learning_rate': 0.004396269099920013, 'subsample': 0.6856835690955101, 'colsample_bytree': 0.8705333287683942, 'min_data_in_leaf': 12}. Best is trial 15 with value: 1975414.629319444.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.5466174715938682, colsample_bytree=0.7249358709419946 will be ignored. Current value: feature_fraction=0.5466174715938682
[LightGBM] [Warning] min_data_in_leaf is set=26, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=26
[LightGBM] [Warning] bagging_fraction is set=0.8823426750516296, subsample=0.8147810856915875 will be ignored. Current value: bagging_fraction=0.8823426750516296
[LightGBM] [Warning] lambda_l2 is set=7.669048442971057, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.669048442971057
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-31 17:23:05,885] Trial 48 finished with value: 2744633.535902318 and parameters: {'n_estimators': 918, 'lambda_l2': 7.669048442971057, 'bagging_fraction': 0.8823426750516296, 'num_leaves': 635, 'feature_fraction': 0.5466174715938682, 'max_depth': 1, 'learning_rate': 0.009535420133002653, 'subsample': 0.8147810856915875, 'colsample_bytree': 0.7249358709419946, 'min_data_in_leaf': 26}. Best is trial 15 with value: 1975414.629319444.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.5090716525747299, colsample_bytree=0.8374747927153042 will be ignored. Current value: feature_fraction=0.5090716525747299
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] bagging_fraction is set=0.9606160500751119, subsample=0.7671816149877119 will be ignored. Current value: bagging_fraction=0.9606160500751119
[LightGBM] [Warning] lambda_l2 is set=6.991151430087696, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.991151430087696
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-31 17:23:22,810] Trial 49 finished with value: 1979262.877532367 and parameters: {'n_estimators': 881, 'lambda_l2': 6.991151430087696, 'bagging_fraction': 0.9606160500751119, 'num_leaves': 705, 'feature_fraction': 0.5090716525747299, 'max_depth': 22, 'learning_rate': 0.008389188896592395, 'subsample': 0.7671816149877119, 'colsample_bytree': 0.8374747927153042, 'min_data_in_leaf': 17}. Best is trial 15 with value: 1975414.629319444.


In [32]:
params = study.best_params
params["objective"] = "regression"
params["metric"] = "rmse"
params["verbosity"] = -1

#Best
params2 = {
    'boosting': 'gbdt', 
    'lambda_l2': 7.068023653932577, 
    'bagging_fraction': 0.6261319161311689, 
    'num_leaves': 151, 
    'feature_fraction': 0.976804739696195, 
    'max_depth': 8, 'learning_rate': 0.01552248392804131, 
    'subsample': 0.4408875142865268, 
    'colsample_bytree': 0.3829262582959811, 
    'min_data_in_leaf': 21,
    "objective": "regression",
    "metric": "rmse",
    "n_estimators": 1000,
    "verbosity": -1,
}

In [33]:
model = lgb.LGBMRegressor(**params2)

model.fit(X, y)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.976804739696195, colsample_bytree=0.3829262582959811 will be ignored. Current value: feature_fraction=0.976804739696195
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Warning] bagging_fraction is set=0.6261319161311689, subsample=0.4408875142865268 will be ignored. Current value: bagging_fraction=0.6261319161311689
[LightGBM] [Warning] lambda_l2 is set=7.068023653932577, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.068023653932577


LightGBMError: GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1

In [10]:
X_pred = model.predict(X[feats])
y_df = pd.DataFrame(y)

y_df['pred'] = X_pred
y_df

,price_doc,pred
0,5.668660e+06,5.404512e+06
1,5.814010e+06,5.381714e+06
2,5.523310e+06,5.252309e+06
3,1.269391e+07,1.140029e+07
4,1.582519e+07,1.407052e+07
...,...,...
30464,7.170610e+06,7.020840e+06
30465,2.422501e+07,2.262009e+07
30466,6.754869e+06,5.344807e+06
30467,1.308151e+07,1.184169e+07


In [11]:
y_df['ratio'] = abs(y_df['pred']/y_df['price_doc'])
y_df.head()

,price_doc,pred,ratio
0,5.668660e+06,5.404512e+06,0.953402
1,5.814010e+06,5.381714e+06,0.925646
2,5.523310e+06,5.252309e+06,0.950935
3,1.269391e+07,1.140029e+07,0.898091
4,1.582519e+07,1.407052e+07,0.889122


In [12]:
# to_remove = y_df[(y_df['ratio'] < 0.5) | (y_df['ratio'] > 2)].index.tolist()
to_remove = y_df[(y_df['ratio'] > 2)].index.tolist()

In [13]:
df2 = pd.read_csv(process_train_path)
dropped_df2 = df2.drop(to_remove)
dropped_df2.to_csv('./processed/dropped_train.csv')

In [29]:
processed_test_df = pd.read_csv(processed_test_path)
processed_test_df_copy = processed_test_df.copy()

processed_test_df = processed_test_df.drop(['id'], axis=1)

In [30]:
pred = model.predict(processed_test_df[feats])
prediction_df = pd.DataFrame({
    'id': processed_test_df_copy['id'],
    'price_doc': pred
})
prediction_df

,id,price_doc
0,30474,5.838440e+06
1,30475,7.952277e+06
2,30476,6.030172e+06
3,30477,6.002089e+06
4,30478,4.889638e+06
...,...,...
7657,38131,8.856842e+06
7658,38132,4.768920e+06
7659,38133,4.591632e+06
7660,38134,5.733782e+06


In [31]:
prediction_df.to_csv(output_path, index=False)

In [ ]:
params